In [ ]:
#Part 1 Camera calibration (intrinsic parameters)
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

#termination criteria
# criteria：迭代停止的模式选择，这是一个含有三个元素的元组型数。格式为（type,max_iter,epsilon） 
#其中，type又有两种选择： 
#—–cv2.TERM_CRITERIA_EPS :精确度（误差）满足epsilon停止。 
#—-cv2.TERM_CRITERIA_MAX_ITER：迭代次数超过max_iter停止。 
#—-cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER，两者合体，任意一个满足结束。
#采用的停止准则是最大循环次数30和最大误差容限0.001
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
#将世界坐标系建在标定板上
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
#print(objp)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane

#input the calibration grid image here.
images = glob.glob('Part1/*.JPG')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6), None) #P418
    
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        #计算角点的精确位置以达到亚像素精度
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria) #P355
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2, ret)
        #cv2.imshow('img', img)
        #cv2.waitKey(500)
        cv2.imwrite("part1_2.jpg", img)

#cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None) #P427
print(mtx) #The intrinsic matrix K
print(dist) #The radial distortion coefficients

mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    #cv2.norm(list1, list2, norm_type, mask)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

print("total error: ", mean_error/len(objpoints))

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
mtx = np.array([[4.30832585e+03, 0.00000000e+00, 2.67770521e+03],
 [0.00000000, 4.33262975e+03, 1.80968008e+03],
 [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
dist = np.array([[-0.06455709,  0.64635224, -0.00222198, -0.00249137, -0.81607703]])

In [ ]:
#part3 compute the relative camera pose
#undistortion
from PIL import Image
from pylab import *
import matplotlib.pyplot as plt
from scipy import optimize
img1 = cv2.imread('Part2/left.JPG')
img2 = cv2.imread('Part2/right.JPG')
h,  w = img1.shape[:2]
dist = np.array([-0.06455709,  0.64635224, -0.00222198, -0.00249137, -0.81607703])
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
dst1 = cv2.undistort(img1, mtx, dist, None, newcameramtx)
dst2 = cv2.undistort(img2, mtx, dist, None, newcameramtx)

plt.imshow(dst1,),plt.show()
plt.imshow(dst2,),plt.show()
cv2.imwrite('dst1.jpg', dst1)
cv2.imwrite('dst2.jpg', dst2)

In [ ]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape[:2]
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

In [ ]:
#find the Fundamental Matrix F
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)
F, mask = cv2.findFundamentalMat(pts1,pts2, cv2.FM_LMEDS)
print(F)

# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

#find the epilines
#Epilines corresponding to the points in first image is drawn on second image
# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3) #未知分为行数3列
img5,img6 = drawlines(gray1,gray2,lines1,pts1,pts2)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img7,img8 = drawlines(gray2,gray1,lines2,pts2,pts1)

cv2.imwrite('7.jpg', img7)
cv2.imwrite('8.jpg', img8)
plt.imshow(img7,),plt.show()
plt.imshow(img8,),plt.show()

In [ ]:
#find the Essiential Matrix F
E, mask = cv2.findEssentialMat(pts1,pts2,mtx)# 18.0, (0, 0))#
#print(mask)
print(E)
R1, R2, t = cv2.decomposeEssentialMat(E)
print(R1)
print(R2)
print(t)

In [ ]:
P21 = np.dot(mtx, np.hstack((R1, t)))
print(P21)
P22 = np.dot(mtx, np.hstack((R1, -t)))
print(P22)
P23 = np.dot(mtx, np.hstack((R2, t)))
print(P23)
P24 = np.dot(mtx, np.hstack((R2, -t)))
print(P24)
P0 = np.array([ [1,0,0,0],
                [0,1,0,0],
                [0,0,1,0] ])
print(mtx)

In [ ]:
for pp1,pp2 in zip(pts1,pts2):
    print(pp1)
    cv2.circle(gray1,tuple(pp1),6,(255,0,0),6)
    cv2.circle(gray2,tuple(pp2),6,(255,0,0),6)

plt.imshow(dst1,),plt.show()
cv2.imwrite('9.jpg', gray1)
cv2.imwrite('10.jpg', gray2)

In [ ]:
d1 = cv2.triangulatePoints(P0,P21,pts1.T, pts2.T)
d1 = d1/d1[3,:]
print(d1)

In [ ]:
d2 = cv2.triangulatePoints(P0,P22,pts1.T, pts2.T)
d2 = d2/d2[3,:]
print(d2)

In [ ]:
d3 = cv2.triangulatePoints(P0,P23,pts1.T, pts2.T)
d3 = d3/d3[3,:]
print(d3)

In [ ]:
d4 = cv2.triangulatePoints(P0,P24,pts1.T, pts2.T)
d4 = d4/d4[3,:]
print(d4)

In [ ]:
d = np.dot(P22, d2)
#d = np.dot(mtx, d3[:3])#4[:3])
d = d/d[2,:]
d = d[:2]
print(d)

In [ ]:
#check if all points matched will
print(d.T)
print(pts2)

In [ ]:
#plot the reprojection picture
df = np.float32(d)
for pp1,pp2 in zip(pts2,df.T):
    #print(pp1)
    cv2.circle(dst2,tuple(pp1),10,(255,0,0),10)
    cv2.circle(dst2,tuple(pp2),6,(0,255,0),6)
plt.imshow(dst2,),plt.show()
cv2.imwrite('reprojection.jpg', dst2)

In [ ]:
#part 4
#set of distance
distance = []
for i in range(20):
    distance.append(5 + i * 37)
print(distance)

In [ ]:
#find the matrix H and get the 20 warped images
dst1 = cv2.undistort(img1, mtx, dist, None, newcameramtx)
dst2 = cv2.undistort(img2, mtx, dist, None, newcameramtx)
for test in distance:
    pOne = np.array([ [1,0,0,0],
                    [0,1,0,0],
                    [0,0,1,0],
                    [0,0,-1,test]])
    pOne = np.float32(pOne)
    print(pOne)
    pTwo = np.array([[ 4.26460842e+03, -9.12316630e+02, -2.59086283e+03,4.26322755e+03],
           [-9.07859026e+02, -4.20372467e+03, -1.88443983e+03,-5.15490408e+02],
           [ 2.32720351e-02,  1.26012045e-02, -9.99649750e-01,-5.85200604e-03],
           [0,0,-1,test]])
    print(pTwo)
    pOneInv = np.linalg.inv(pOne)
    print(pOneInv)
    H = pTwo * pOneInv
    #H = H[:3, :3]
    #print(H)
    H = R2.T - (t) * (np.array([0, 0, -1]).T) / test
    print(H)
    mat = mtx * H * np.linalg.inv(mtx)

    print(mat)
    h,  w = img1.shape[:2]
    warped = cv2.warpPerspective(dst2,mat,(w,h))
    #test = np.dot(mat, [783.37919437, 2518.51272543])
    plt.imshow(dst2),plt.show()
    plt.imshow(warped,),plt.show()
    cv2.imwrite('warped1/warped' + str(test) + '.jpg', warped)

In [ ]:
dst1 = cv2.undistort(img1, mtx, dist, None, newcameramtx)
dst1G=cv2.cvtColor(dst1, cv2.COLOR_BGR2GRAY)
warped5 = np.float32(cv2.imread('warped1/warped5.jpg'))
warped5G=cv2.cvtColor(warped5, cv2.COLOR_BGR2GRAY)
warped42 = np.float32(cv2.imread('warped1/warped42.jpg'))
warped42G=cv2.cvtColor(warped42, cv2.COLOR_BGR2GRAY)
warped79 = np.float32(cv2.imread('warped1/warped79.jpg'))
warped79G=cv2.cvtColor(warped79, cv2.COLOR_BGR2GRAY)
warped116 = np.float32(cv2.imread('warped1/warped116.jpg'))
warped116G=cv2.cvtColor(warped116, cv2.COLOR_BGR2GRAY)
warped153 = np.float32(cv2.imread('warped1/warped153.jpg'))
warped153G=cv2.cvtColor(warped153, cv2.COLOR_BGR2GRAY)
warped190 = np.float32(cv2.imread('warped1/warped190.jpg'))
warped190G=cv2.cvtColor(warped190, cv2.COLOR_BGR2GRAY)
warped227 = np.float32(cv2.imread('warped1/warped227.jpg'))
warped227G=cv2.cvtColor(warped227, cv2.COLOR_BGR2GRAY)
warped264 = np.float32(cv2.imread('warped1/warped264.jpg'))
warped264G=cv2.cvtColor(warped264, cv2.COLOR_BGR2GRAY)
warped301 = np.float32(cv2.imread('warped1/warped301.jpg'))
warped301G=cv2.cvtColor(warped301, cv2.COLOR_BGR2GRAY)
warped338 = np.float32(cv2.imread('warped1/warped338.jpg'))
warped338G=cv2.cvtColor(warped338, cv2.COLOR_BGR2GRAY)
warped375 = np.float32(cv2.imread('warped1/warped375.jpg'))
warped375G=cv2.cvtColor(warped375, cv2.COLOR_BGR2GRAY)
warped412 = np.float32(cv2.imread('warped1/warped412.jpg'))
warped412G=cv2.cvtColor(warped412, cv2.COLOR_BGR2GRAY)
warped449 = np.float32(cv2.imread('warped1/warped449.jpg'))
warped449G=cv2.cvtColor(warped449, cv2.COLOR_BGR2GRAY)
warped486 = np.float32(cv2.imread('warped1/warped486.jpg'))
warped486G=cv2.cvtColor(warped486, cv2.COLOR_BGR2GRAY)
warped523 = np.float32(cv2.imread('warped1/warped523.jpg'))
warped523G=cv2.cvtColor(warped523, cv2.COLOR_BGR2GRAY)
warped560 = np.float32(cv2.imread('warped1/warped560.jpg'))
warped560G=cv2.cvtColor(warped560, cv2.COLOR_BGR2GRAY)
warped597 = np.float32(cv2.imread('warped1/warped597.jpg'))
warped597G=cv2.cvtColor(warped597, cv2.COLOR_BGR2GRAY)
warped634 = np.float32(cv2.imread('warped1/warped634.jpg'))
warped634G=cv2.cvtColor(warped634, cv2.COLOR_BGR2GRAY)
warped671 = np.float32(cv2.imread('warped1/warped671.jpg'))
warped671G=cv2.cvtColor(warped671, cv2.COLOR_BGR2GRAY)
warped708 = np.float32(cv2.imread('warped1/warped708.jpg'))
warped708G=cv2.cvtColor(warped708, cv2.COLOR_BGR2GRAY)
print(warped5G[1000, 2000])

In [ ]:
D = []

In [ ]:
D1 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D1[i][j] = abs(warped5G[i][j] - dst1G[i][j])
D.append(D1)
print(D1[1000][500])

In [ ]:
D2 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D2[i][j] = abs(warped42G[i][j] - dst1G[i][j])
print(D2[1000][500])

In [ ]:
D3 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D3[i][j] = abs(warped79G[i][j] - dst1G[i][j])
print(D3[1000][500])

In [ ]:
D4 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D4[i][j] = abs(warped116G[i][j] - dst1G[i][j])
print(D4[1000][500])

In [ ]:
D5 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D5[i][j] = abs(warped153G[i][j] - dst1G[i][j])
print(D5[1000][500])

In [ ]:
D6 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D6[i][j] = abs(warped190G[i][j] - dst1G[i][j])
print(D6[1000][500])

In [ ]:
D7 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D7[i][j] = abs(warped227G[i][j] - dst1G[i][j])
print(D7[1000][500])

In [ ]:
D8 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D8[i][j] = abs(warped264G[i][j] - dst1G[i][j])
print(D8[1000][500])

In [ ]:
D9 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D9[i][j] = abs(warped301G[i][j] - dst1G[i][j])
print(D9[1000][500])

In [ ]:
D10 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D10[i][j] = abs(warped338G[i][j] - dst1G[i][j])
print(D10[1000][500])

In [ ]:
D11 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D11[i][j] = abs(warped375G[i][j] - dst1G[i][j])
print(D11[1000][500])

In [ ]:
D12 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D12[i][j] = abs(warped412G[i][j] - dst1G[i][j])
print(D12[1000][500])

In [ ]:
D13 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D13[i][j] = abs(warped449G[i][j] - dst1G[i][j])
print(D8[1000][500])

In [ ]:
D14 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D14[i][j] = abs(warped486G[i][j] - dst1G[i][j])
print(D14[1000][500])

In [ ]:
D15 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D15[i][j] = abs(warped523G[i][j] - dst1G[i][j])
print(D15[1000][500])

In [ ]:
D16 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D16[i][j] = abs(warped560G[i][j] - dst1G[i][j])
print(D16[1000][500])

In [ ]:
D17 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D17[i][j] = abs(warped597G[i][j] - dst1G[i][j])
print(D17[1000][500])

In [ ]:
D18 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D18[i][j] = abs(warped634G[i][j] - dst1G[i][j])
print(D18[1000][500])

In [ ]:
D19 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D19[i][j] = abs(warped671G[i][j] - dst1G[i][j])
print(D19[1000][500])

In [ ]:
D20 = [[0] * (w) for _ in range(h)]
for i in range(h):
    for j in range(w):
        D20[i][j] = abs(warped708G[i][j] - dst1G[i][j])
print(D20[1000][500])

In [ ]:
D = []
D.append(D1)
D.append(D2)
D.append(D3)
D.append(D4)
D.append(D5)
D.append(D6)
D.append(D7)
D.append(D8)
D.append(D9)
D.append(D10)
D.append(D11)
D.append(D12)
D.append(D13)
D.append(D14)
D.append(D15)
D.append(D16)
D.append(D17)
D.append(D18)
D.append(D19)
D.append(D20)

In [ ]:
DC = np.array(D)

In [ ]:
for i in range(20):
    DC[i] = cv2.boxFilter(DC[i], -1, (20, 20))

In [ ]:
#find the depth assigned pixel and plot the depth image
res = [[0] * w for _ in range(h)]
for i in range(len(DC[0])):
    for j in range(len(DC[0][0])):
        minI = float('inf')
        minD = 0
        for n in range(20):
            if DC[n][i][j] < minI:
                minI = DC[n][i][j]
                minD = n
                res[i][j] = (20 - n) * 255 // 20
plt.imshow(res),plt.show()